## New patient classification with incomplete omics profiles

## Import packages and IntegrAO code

In [1]:
import numpy as np
import pandas as pd
import snf
from sklearn.cluster import spectral_clustering
from sklearn.metrics import v_measure_score
import matplotlib.pyplot as plt

import sys
import os
import argparse
import torch

import umap
from sklearn.model_selection import train_test_split

/home/jma/anaconda3/envs/integrAO/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Add the parent directory of "integrao" to the Python path
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from integrao.dataset import GraphDataset
from integrao.main import dist2
from integrao.integrater import integrao_integrater, integrao_predictor

## Set hyperparameters

In [3]:
torch.cuda.is_available()

True

In [4]:
# Hyperparameters
neighbor_size = 20
embedding_dims = 64
fusing_iteration = 30
normalization_factor = 1.0
alighment_epochs = 1000
beta = 1.0
mu = 0.5


dataset_name = 'cancer_omics_prediction'
cluster_number = 15

In [5]:
# create result dir
result_dir = os.path.join(
    module_path, "results/{}".format(dataset_name)
)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

## Read data

In [6]:
testdata_dir = os.path.join(module_path, "data/omics/")

methyl_ = os.path.join(testdata_dir, "omics1.txt")
expr_ = os.path.join(testdata_dir, "omics2.txt")
protein_ = os.path.join(testdata_dir, "omics3.txt")
truelabel = os.path.join(testdata_dir, "clusters.txt")


methyl = pd.read_csv(methyl_, index_col=0, delimiter="\t")
expr = pd.read_csv(expr_, index_col=0, delimiter="\t")
protein = pd.read_csv(protein_, index_col=0, delimiter="\t")
truelabel = pd.read_csv(truelabel, index_col=0, delimiter="\t")

methyl = np.transpose(methyl)
expr = np.transpose(expr)
protein = np.transpose(protein)
print(methyl.shape)
print(expr.shape)
print(protein.shape)
print(truelabel.shape)
print("finish loading data!")

(500, 367)
(500, 131)
(500, 160)
(500, 2)
finish loading data!


## Random stratified-subsample 80%-20% samples to simulate the senario of incomplete omics dataset


In [7]:
truelabel

,subjects,cluster.id
1,subject1,6
2,subject2,7
3,subject3,9
4,subject4,6
5,subject5,4
...,...,...
496,subject496,1
497,subject497,14
498,subject498,4
499,subject499,1


In [8]:
common_patient = methyl.index
y = truelabel['cluster.id'].tolist()

X_train, X_test, y_train, y_test = train_test_split(common_patient, y, stratify=y, test_size=0.2)

# get the reference and query data
methyl_ref = methyl.loc[X_train]
expr_ref = expr.loc[X_train]
protein_ref = protein.loc[X_train]

methyl_query = methyl.loc[X_test]
expr_query = expr.loc[X_test]
protein_query = protein.loc[X_test]

## Now let's intergrate the reference data 

In [9]:
# Initialize integrater
integrater = integrao_integrater(
    [methyl_ref, expr_ref, protein_ref],
    dataset_name,
    modalities_name_list=["methyl", "expr", "protein"],   # used for naming the incomplete modalities during new sample inference
    neighbor_size=neighbor_size,
    embedding_dims=embedding_dims,
    fusing_iteration=fusing_iteration,
    normalization_factor=normalization_factor,
    alighment_epochs=alighment_epochs,
    beta=beta,
    mu=mu,
)
# data indexing
fused_networks = integrater.network_diffusion()
embeds_final, S_final, model = integrater.unsupervised_alignment()

# save the model for fine-tuning
torch.save(model.state_dict(), os.path.join(result_dir, "model.pth"))

Start indexing input expression matrices!
Common sample between view0 and view1: 400
Common sample between view0 and view2: 400
Common sample between view1 and view2: 400
Neighbor size: 20
Start applying diffusion!
Diffusion ends! Times: 4.388705015182495s
Starting unsupervised exmbedding extraction!
Dataset 0: (400, 367)
Dataset 1: (400, 131)
Dataset 2: (400, 160)
epoch 0: loss 27.789127349853516, align_loss:0.744084
epoch 100: loss 19.32496452331543, align_loss:0.101542
epoch 200: loss 0.7247133255004883, align_loss:0.061813
epoch 300: loss 0.7239326238632202, align_loss:0.061128
epoch 400: loss 0.7230430245399475, align_loss:0.060395
epoch 500: loss 0.7220683693885803, align_loss:0.059626
epoch 600: loss 0.7210268974304199, align_loss:0.058864
epoch 700: loss 0.7199146747589111, align_loss:0.058144
epoch 800: loss 0.718734085559845, align_loss:0.057389
epoch 900: loss 0.7174885272979736, align_loss:0.056649
Manifold alignment ends! Times: 7.765907526016235s


In [10]:
labels = spectral_clustering(S_final, n_clusters=cluster_number)

# select from truelabel based on the 'subjects' column in embeds_final
truelabel_filtered = truelabel[truelabel['subjects'].isin(embeds_final.index)]
truelabel_filtered = truelabel_filtered.sort_values('subjects')['cluster.id'].tolist()

score_all = v_measure_score(truelabel_filtered, labels)
print("IntegrAO for clustering reference 400 samples NMI score: ", score_all)

IntegrAO for clustering reference 400 samples NMI score:  1.0


## Now to perform fine-tuning using on the ground true labels

In [11]:
truelabel_sub = truelabel[truelabel['subjects'].isin(embeds_final.index)]
truelabel_sub = truelabel_sub.set_index('subjects')

# minus 1 for the cluster id to avoid CUDA error
truelabel_sub['cluster.id'] = truelabel_sub['cluster.id'] - 1
truelabel_sub

,cluster.id
subjects,
subject1,5
subject2,6
subject3,8
subject4,5
subject5,3
...,...
subject496,0
subject497,13
subject498,3


In [13]:
embeds_final, S_final, model, preds = integrater.classification_finetuning(truelabel_sub, result_dir, finetune_epochs=800)

Starting supervised fineting!
Dataset 0: (400, 367)
Dataset 1: (400, 131)
Dataset 2: (400, 160)
IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
epoch 0: loss 3.224135637283325, kl_loss:0.669090, align_loss:0.052747, clf_loss:2.502299
epoch 100: loss 0.6

In [14]:
torch.save(model.state_dict(), os.path.join(result_dir, "model_integrao_supervised.pth"))

## Now to perform inference on query data

In [15]:
# Network fusion for the whole graph
predictor = integrao_predictor(
    [methyl, expr, protein],
    dataset_name,
    modalities_name_list=["methyl", "expr", "protein"], 
    neighbor_size=neighbor_size,
    embedding_dims=embedding_dims,
    fusing_iteration=fusing_iteration,
    normalization_factor=normalization_factor,
    alighment_epochs=alighment_epochs,
    beta=beta,
    mu=mu,
    num_classes=cluster_number,
)
# data indexing
fused_networks = predictor.network_diffusion()

Start indexing input expression matrices!
Common sample between view0 and view1: 500
Common sample between view0 and view2: 500
Common sample between view1 and view2: 500
Neighbor size: 20
Start applying diffusion!
Diffusion ends! Times: 5.840329647064209s


In [16]:
from sklearn.metrics import accuracy_score, f1_score

# helper function to get the metrics on test set
def get_metrics(preds, preds_index, X_test, y_test):

    pred_df = pd.DataFrame(data=preds, index=preds_index)
    pred_df_test = pred_df.loc[X_test]

    # add 1 back to the cluster id
    pred_df_test = pred_df_test + 1

    f1_micro = f1_score(y_test, pred_df_test, average='micro')
    f1_weighted = f1_score(y_test, pred_df_test, average='weighted')
    acc = accuracy_score(y_test, pred_df_test)

    return f1_micro, f1_weighted, acc


## Using one modalities

In [17]:
model_path = os.path.join(result_dir, "model_integrao_supervised.pth")

In [18]:
# for methyl
preds = predictor.inference_supervised(model_path, new_datasets=[methyl], modalities_names=["methyl"])

f1_micro, f1_weight, acc = get_metrics(preds, methyl.index, X_test, y_test)

print("methyl f1_micro: ", f1_micro)
print("methyl f1_weight: ", f1_weight)
print("methyl acc: ", acc)

IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
methyl f1_micro:  1.0
methyl f1_weight:  1.0
methyl acc:  1.0


In [20]:
# for expr
preds = predictor.inference_supervised(model_path, new_datasets=[expr], modalities_names=["expr"])

f1_micro, f1_weight, acc = get_metrics(preds, expr.index, X_test, y_test)

print("expr f1_micro: ", f1_micro)
print("expr f1_weight: ", f1_weight)
print("expr acc: ", acc)

IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
expr f1_micro:  0.99
expr f1_weight:  0.989047619047619
expr acc:  0.99


In [21]:
# for protein
preds = predictor.inference_supervised(model_path, new_datasets=[protein], modalities_names=["protein"])

f1_micro, f1_weight, acc = get_metrics(preds, protein.index, X_test, y_test)


print("protein f1_micro: ", f1_micro)
print("protein f1_weight: ", f1_weight)
print("protein acc: ", acc)

IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
protein f1_micro:  1.0
protein f1_weight:  1.0
protein acc:  1.0


## Two modalities

In [22]:
# methyl and expr
preds = predictor.inference_supervised(model_path, new_datasets=[methyl, expr], modalities_names=["methyl", "expr"])

f1_micro, f1_weight, acc = get_metrics(preds, methyl.index, X_test, y_test)

print("methyl+expr f1_micro: ", f1_micro)
print("methyl+expr f1_weight: ", f1_weight)
print("methyl+expr acc: ", acc)

IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
methyl+expr f1_micro:  1.0
methyl+expr f1_weight:  1.0
methyl+expr acc:  1.0


In [23]:
# methyl and protein
preds = predictor.inference_supervised(model_path, new_datasets=[methyl, protein], modalities_names=["methyl", "protein"])

f1_micro, f1_weight, acc = get_metrics(preds, methyl.index, X_test, y_test)

print("methyl+protein f1_micro: ", f1_micro)
print("methyl+protein f1_weight: ", f1_weight)
print("methyl+protein acc: ", acc)

IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
methyl+protein f1_micro:  1.0
methyl+protein f1_weight:  1.0
methyl+protein acc:  1.0


In [24]:
# expr and protein
preds = predictor.inference_supervised(model_path, new_datasets=[expr, protein], modalities_names=["expr", "protein"])

f1_micro, f1_weight, acc = get_metrics(preds, expr.index, X_test, y_test)

print("expr+protein f1_micro: ", f1_micro)
print("expr+protein f1_weight: ", f1_weight)
print("expr+protein acc: ", acc)

IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
expr+protein f1_micro:  1.0
expr+protein f1_weight:  1.0
expr+protein acc:  1.0


## Three modalities

In [25]:
# methyl, expr and protein
preds = predictor.inference_supervised(model_path, new_datasets=[methyl, expr, protein], modalities_names=["methyl", "expr", "protein"])

f1_micro, f1_weight, acc = get_metrics(preds, methyl.index, X_test, y_test)

print("methyl+expr+protein f1_micro: ", f1_micro)
print("methyl+expr+protein f1_weight: ", f1_weight)
print("methyl+expr+protein acc: ", acc)

IntegrAO(
  (feature): ModuleList(
    (0): GraphSAGE(367, 64, num_layers=2)
    (1): GraphSAGE(131, 64, num_layers=2)
    (2): GraphSAGE(160, 64, num_layers=2)
  )
  (feature_show): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (pred_head): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Linear(in_features=32, out_features=15, bias=True)
  )
)
Loaded pre-trained model with success.
methyl+expr+protein f1_micro:  1.0
methyl+expr+protein f1_weight:  1.0
methyl+expr+protein acc:  1.0
